**Import libraries**

In [34]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

import os

import matplotlib.pyplot as plt

import numpy as np

import pandas as pd

from tensorflow.keras.applications import ResNet50

from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import regularizers

from tensorflow.keras.regularizers import l2

In [35]:
dataset_path = '/kaggle/input/cropped-dataset'

num_classes = 105

image_paths = []

labels = []



for class_name in os.listdir(dataset_path):

    class_path = os.path.join(dataset_path, class_name)

    if os.path.isdir(class_path):

        for img_name in os.listdir(class_path):

            img_path = os.path.join(class_path, img_name)

            image_paths.append(img_path)

            labels.append(class_name)


In [36]:
labels = np.array(labels)

label_encoder = LabelEncoder()
numeric_labels = label_encoder.fit_transform(labels)

In [37]:
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, numeric_labels, test_size=0.1, stratify=numeric_labels, random_state=42
)
# val_paths, test_paths, val_labels, test_labels = train_test_split(
#     test_val_paths, test_val_labels, test_size=0.5, stratify=test_val_labels, random_state=42
# )

In [38]:
def load_image_dataset(image_paths, labels, img_height=224, img_width=224, batch_size=32):
    def process_image(file_path, label):
        image = tf.io.read_file(file_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [img_height, img_width])
        image = tf.cast(image, tf.float32) / 255.0
        return image, label

    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(lambda x, y: process_image(x, y), num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [39]:
train_dataset = load_image_dataset(train_paths, train_labels)

val_dataset = load_image_dataset(val_paths, val_labels)

# test_dataset = load_image_dataset(test_paths, test_labels)

In [40]:
print('Number of Train samples:', len(train_paths))

print('Number of Validation samples:', len(val_paths))

# print('Number of Test samples:', len(test_paths))

Number of Train samples: 15819
Number of Validation samples: 1758


In [41]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [42]:
x = base_model.output

x = Flatten()(x)

x = Dense(1024, activation='relu')(x)

x = BatchNormalization()(x)

x = Dropout(0.2)(x)

x = Dense(512, activation='relu')(x)

x = BatchNormalization()(x)

x = Dropout(0.3)(x)

x = Dense(256, activation='relu')(x)

x = BatchNormalization()(x)

x = Dropout(0.4)(x)

output_layer = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output_layer)

In [43]:
model.compile(optimizer=Adam(learning_rate=0.0001),

              loss='sparse_categorical_crossentropy',

              metrics=['accuracy'])


In [44]:
history = model.fit(

    train_dataset,  

    validation_data=val_dataset, 

    epochs=10,  

    batch_size=32  

)

Epoch 1/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 160s 213ms/step - accuracy: 0.0568 - loss: 4.9100 - val_accuracy: 0.0148 - val_loss: 5.0128
Epoch 2/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 81s 163ms/step - accuracy: 0.3484 - loss: 2.7727 - val_accuracy: 0.4619 - val_loss: 2.4074
Epoch 3/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 81s 163ms/step - accuracy: 0.6713 - loss: 1.4386 - val_accuracy: 0.7412 - val_loss: 1.2366
Epoch 4/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 81s 163ms/step - accuracy: 0.8852 - loss: 0.6394 - val_accuracy: 0.7713 - val_loss: 1.0623
Epoch 5/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 81s 164ms/step - accuracy: 0.9794 - loss: 0.2365 - val_accuracy: 0.8157 - val_loss: 0.8329
Epoch 6/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 82s 165ms/step - accuracy: 0.9927 - loss: 0.1076 - val_accuracy: 0.8276 - val_loss: 0.7297
Epoch 7/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 82s 165ms/step - accuracy: 0.9988 - loss: 0.0565 - val_accuracy: 0.8424 - val_loss: 0.6626
Epoch 8/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 81s 164ms/step - accuracy: 0.9997 - loss: 